In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import sys
import random
import glob
import pickle
import itertools
import io
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas
import time
from tqdm import tqdm

import sklearn

import tensorflow as tf

# utils lib for the tf dataset
import tf_data

In [4]:
data_path = {"ele_match": "/eos/user/r/rdfexp/ecal/cluster/output_deepcluster_dumper/windows_data/electrons/recordio_allinfo_v2/training/calo_matched/*.proto",
             "gamma_match": "/eos/user/r/rdfexp/ecal/cluster/output_deepcluster_dumper/windows_data/gammas/recordio_allinfo_v2/training/calo_matched/*.proto",
             "ele_nomatch": "/eos/user/r/rdfexp/ecal/cluster/output_deepcluster_dumper/windows_data/electrons/recordio_allinfo_v2/training/no_calo_matched/*.proto",
             "gamma_nomatch": "/eos/user/r/rdfexp/ecal/cluster/output_deepcluster_dumper/windows_data/gammas/recordio_allinfo_v2/training/no_calo_matched/*.proto"
            }

In [22]:
features = ["cluster_ieta","cluster_iphi","cluster_iz",
            "cluster_deta", "cluster_dphi",
            "en_cluster","et_cluster", 
            "cl_swissCross", "cl_nxtals", "cl_etaWidth","cl_phiWidth"]
#features = ["et_cluster"]

In [6]:
_load_n_items = 10
_batch_size = 5
_nepochs = 100
_ntrain = 10000
_nval = 1000

In [23]:
dataset = tf.data.TFRecordDataset(tf.io.gfile.glob(data_path["ele_match"]))

dataset =  dataset.batch(_batch_size).map(
                lambda el: tf_data.parse_windows_batch(el, True),
                num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)

dataset = tf_data.cluster_features_and_hits(dataset,features)

In [24]:
a = next(iter(dataset.take(1)))

In [25]:
cl_X, cl_hits, is_seed, in_sc,  cl_features = a

In [26]:
cl_X

<tf.Tensor: shape=(5, 7, 12), dtype=float32, numpy=
array([[[ 8.50000000e+01,  3.34000000e+02,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  1.14316826e+02,
          4.98746910e+01,  4.59328473e-01,  1.10000000e+01,
          6.94015669e-03,  1.02384873e-02,  1.00000000e+00],
        [ 8.50000000e+01,  3.28000000e+02,  0.00000000e+00,
          3.03769112e-03,  9.30574536e-02,  3.05592632e+00,
          1.32961380e+00,  6.58439577e-01,  2.00000000e+00,
          3.01638572e-03,  8.66006128e-03,  0.00000000e+00],
        [ 7.80000000e+01,  3.06000000e+02,  0.00000000e+00,
         -1.15920544e-01,  4.84164059e-01,  4.16763902e-01,
          2.01541260e-01,  1.00000000e+00,  1.00000000e+00,
          2.28552567e-03,  1.14631653e-03,  0.00000000e+00],
        [ 7.80000000e+01,  1.00000000e+00,  0.00000000e+00,
         -1.16546869e-01, -4.76787448e-01,  3.97348166e-01,
          1.92257434e-01,  1.00000000e+00,  1.00000000e+00,
          2.25436781e-03,  1.12272799e-03,  0